In [84]:
using FileIO, MeshIO, Meshes, MeshViz
using GLMakie

┌ Info: Precompiling MeshIO [7269a6da-0436-5bbc-96c2-40638cbb6118]
└ @ Base loading.jl:1313
┌ Info: Precompiling Meshes [eacbb407-ea5a-433e-ab97-5258b1ca43fa]
└ @ Base loading.jl:1313
┌ Info: Precompiling MeshViz [9ecf9c4f-6e5a-4b5e-83ae-06f2c7a661d8]
└ @ Base loading.jl:1313
┌ Info: Precompiling GLMakie [e9467ef8-e4e7-5192-8a1a-b1aee30e663a]
└ @ Base loading.jl:1313


In [130]:
cat_mesh_ = load(joinpath("..", "data", "cat-00.off"))
lion_mesh_ = load(joinpath("..", "data", "lion-00.off"));

In [131]:
import GeometryBasics
function Base.convert(::Type{<:SimpleMesh}, m::GeometryBasics.Mesh)
    points = [Meshes.Point.(p...) for p in m.position]
    connec = [connect(Tuple(Int(offi.i)+1 for offi in c)) for c in GeometryBasics.faces(m)]
    SimpleMesh(points, connec)
end

In [132]:
cat_mesh = convert(SimpleMesh, cat_mesh_)
lion_mesh = convert(SimpleMesh, lion_mesh_);

In [133]:
# viz(cat_mesh, showfacets =true) |> display

In [134]:
function double_plot(mesh1, mesh2; color1=:yellow, color2=:yellow, showfacets=false, landmarks=undef)
    f = Figure()
    ax1 = Axis3(f[1, 1], aspect = :data)
    viz!(mesh1,vertexcolor=color1,showfacets=showfacets, showvertices=true)
    ax2 = Axis3(f[1, 2], aspect = :data)
    viz!(mesh2, vertexcolor=color2, showfacets=showfacets, showvertices=true)
    return f, [ax1, ax2]
end

double_plot (generic function with 1 method)

In [135]:
fig, fig_axes = double_plot(cat_mesh, lion_mesh; color1=1:length(cat_mesh.points), color2=1:length(lion_mesh.points), showfacets=false)
fig |> display

GLMakie.Screen(...)

In [136]:
# load landmark
all_landmarks = open(joinpath("..", "data", "landmarks.txt"),"r") do datafile
    [parse.(Int, split(line)) for line in eachline(datafile)]
end
landmarks = [p .+ 1 for p in all_landmarks[1:5]]

5-element Vector{Vector{Int64}}:
 [3178, 1686]
 [7179, 4911]
 [6566, 4647]
 [5473, 4276]
 [2384, 1384]

In [137]:
# visualize landmarks
meshes = [cat_mesh, lion_mesh]
for i in [1,2]
    cat_landmarks_x = [meshes[i].points[pp[i]].coords[1] for pp in landmarks]
    cat_landmarks_y = [meshes[i].points[pp[i]].coords[2] for pp in landmarks]
    cat_landmarks_z = [meshes[i].points[pp[i]].coords[3] for pp in landmarks]
    meshscatter!(fig_axes[i], cat_landmarks_x,cat_landmarks_y,cat_landmarks_z, markersize = 0.01, color = 1:length(landmarks))
end

fig |> display

GLMakie.Screen(...)

In [1]:
using PyCall

In [2]:
py"""
import numpy as np

from pyFM.mesh import TriMesh
from pyFM.functional import FunctionalMapping

def get_fmmodel():
    mesh1 = TriMesh('../data/cat-00.off')
    mesh2 = TriMesh('../data/lion-00.off')

    process_params = {
        'n_ev': (35,35),  # Number of eigenvalues on source and Target
        'landmarks': np.loadtxt('../data/landmarks.txt',dtype=int)[:5],  # loading 5 landmarks
        'subsample_step': 5,  # In order not to use too many descriptors
        'descr_type': 'WKS',  # WKS or HKS
    }

    model = FunctionalMapping(mesh1,mesh2)
    model.preprocess(**process_params,verbose=True)
    return model
"""

In [3]:
py_FM_model = py"get_fmmodel"()


Computing Laplacian spectrum
Computing 200 eigenvectors
	Done in 1.71 s
Computing 200 eigenvectors
	Done in 1.25 s

Computing descriptors
	Normalizing descriptors

	120 out of 600 possible descriptors kept


PyObject <pyFM.functional.FunctionalMapping object at 0x00000000651F06A0>

In [49]:
using SparseArrays
const scipy_sparse_find = pyimport("scipy.sparse")["find"]
# convert py sparse matrix to a Julia sparse array
function mysparse(Apy::PyObject)
    IA, JA, SA = scipy_sparse_find(Apy)
    return sparse(Int[i+1 for i in IA], Int[i+1 for i in JA], SA)
end

mysparse (generic function with 1 method)

In [6]:
pymesh1 = py_FM_model.mesh1
pymesh2 = py_FM_model.mesh2
const k1 = py_FM_model.k1
const k2 =  py_FM_model.k2

35

In [32]:
py"""
import meshplot as mp
mp.offline()

def plot_mesh(myMesh,cmap=None,file_name="test.html"):
    p = mp.plot(myMesh.vertlist, myMesh.facelist,c=cmap)
    p.save(file_name)
    
def double_plot(myMesh1,myMesh2,cmap1=None,cmap2=None):
    d = mp.subplot(myMesh1.vertlist, myMesh1.facelist, c=cmap1, s=[2, 2, 0])
    mp.subplot(myMesh2.vertlist, myMesh2.facelist, c=cmap2, s=[2, 2, 1], data=d)
    d.save('test.html')

def visu(vertices):
    min_coord,max_coord = np.min(vertices,axis=0,keepdims=True),np.max(vertices,axis=0,keepdims=True)
    cmap = (vertices-min_coord)/(max_coord-min_coord)
    return cmap
"""

$\newcommand{\RR}{\mathbb{R}}$
$\newcommand{\uargmin}[1]{\underset{#1}{\text{argmin}}\;}$
$\newcommand{\uargmax}[1]{\underset{#1}{\text{argmax}}\;}$
$\def\*#1{\mathbf{#1}}$

The fitting optimization problem is
\begin{equation}
\uargmin{\*C\in\RR^{k_2\times k_1}} \mu_{descr}\|\*C\*A - \*B\|^2 + \mu_{lap}\|\*C\Delta_1 - \Delta_2\*C\|^2 + \mu_{\text{descr comm}}\sum_i \|\*C\Gamma_1^i - \Gamma_2^i\*C\|^2 + \mu_{\text{orient}}\sum_i \|\*C\Lambda_1^i - \Lambda_2^i\*C\|^2
\end{equation}

with $\Gamma_1^i$ and $\Gamma_2^i$ [multipliative operators](http://www.lix.polytechnique.fr/~maks/papers/fundescEG17.pdf) associated to the $i$-th descriptors, $\Lambda_1^i$ and $\Lambda_2^i$ [orientation preserving operators](https://arxiv.org/abs/1806.04455) associated to the $i$-th descriptors

In [50]:
descr_μ = 1e0 # descr_preservation
lap_μ = 1e-2 # LB_commutation
descr_comm_μ = 1e-1 # oplist_commutation

orient_μ = 0

0

In [52]:
# Compute multiplicative operators associated to each descriptor
"""
Compute the multiplication operators associated with the descriptors

Output
======
operators : n_descr long list of ((k1,k1),(k2,k2)) operators.
"""
function compute_descr_op(pymesh1, pymesh2, descr1, descr2, k1, k2)
    pinv1 = pymesh1.eigenvectors[:, 1:k1]' * mysparse(pymesh1.A)  # (k1,n)
    pinv2 = pymesh2.eigenvectors[:, 1:k2]' * mysparse(pymesh2.A)  # (k2,n)
    list_descr = [
                  (pinv1 * (descr1[:, i] .* pymesh1.eigenvectors[:, 1:k1]),
                   pinv2 * (descr2[:, i] .* pymesh2.eigenvectors[:, 1:k2])
                   )
                  for i in 1:size(descr1, 2)
                  ]
    return list_descr
end

compute_descr_op

In [53]:
list_descr = compute_descr_op(pymesh1, pymesh2, py_FM_model.descr1, py_FM_model.descr2, k1, k2);  
# (n_descr, ((k1,k1), (k2,k2)) )

In [54]:
# Compute multiplicative operators associated to each descriptor
# skip for now

In [55]:
# Compute the squared differences between eigenvalues for LB commutativity
using LinearAlgebra

ev_sqdiff = (pymesh1.eigenvalues[1:k1]' .- pymesh1.eigenvalues[1:k2]).^2
ev_sqdiff /= LinearAlgebra.norm(ev_sqdiff);

In [116]:
viz_fm_matrix(ev_sqdiff) |> display

GLMakie.Screen(...)

In [56]:
# initial map guess
# Sets the equivalence between the constant functions
x0 = zeros(Float64, k2, k1)

ev_sign = sign.(pymesh1.eigenvectors[1,1]*pymesh2.eigenvectors[1,1])
area_ratio = sqrt.(pymesh2.area/pymesh1.area)

x0[:,1] = zeros(k2)
x0[1,1] = ev_sign * area_ratio

1.2425867898388572

In [57]:
function viz_fm_matrix(FM)
    hfig = Figure()
    ax, hm = heatmap(hfig[1, 1], FM)
    Colorbar(hfig[1, 2], hm)
    hfig
end

viz_fm_matrix (generic function with 1 method)

In [117]:
viz_fm_matrix(x0) |> display

GLMakie.Screen(...)

## optimization objectives

In [120]:
"""
Compute the descriptor preservation constraint

Parameters
=========
C      : (K2,K1) Functional map
descr1 : (K1,p) descriptors on first basis
descr2 : (K2,p) descriptros on second basis

Output
========
energy : descriptor preservation squared norm
"""
function descr_preservation(C, descr1, descr2)
    0.5 * sum((C * descr1 - descr2).^2)
end

"""
Compute the gradient of the descriptor preservation constraint

Parameters
---------------------
C      : (K2,K1) Functional map
descr1 : (K1,p) descriptors on first basis
descr2 : (K2,p) descriptros on second basis

Output
---------------------
gradient : gradient of the descriptor preservation squared norm
"""
function descr_preservation_grad(C, descr1, descr2)
    return (C * descr1 - descr2) * descr1'
end

descr_preservation_grad

In [59]:
"""
Compute the LB commutativity constraint

Parameters
===========
C      : (K2,K1) Functional map
ev_sqdiff : (K2,K1) [normalized] matrix of squared eigenvalue differences

Output
======
energy : (float) LB commutativity squared norm
"""
function LB_commutation(C, ev_sqdiff)
    0.5 * sum(C.^2 * ev_sqdiff)
end

"""
Compute the gradient of the LB commutativity constraint

Parameters
---------------------
C         : (K2,K1) Functional map
ev_sqdiff : (K2,K1) [normalized] matrix of squared eigenvalue differences

Output
---------------------
gradient : (K2,K1) gradient of the LB commutativity squared norm
"""
function LB_commutation_grad(C, ev_sqdiff)
    return C * ev_sqdiff
end

LB_commutation_grad

In [60]:
"""
Compute the operator commutativity constraint.
Can be used with descriptor multiplication operator

Parameters
---------------------
C   : (K2,K1) Functional map
op1 : (K1,K1) operator on first basis
op2 : (K2,K2) descriptros on second basis

Output
---------------------
energy : (float) operator commutativity squared norm
"""
function op_commutation(C, op1, op2)
    return 0.5 * sum((C * op1 - op2 * C).^2)
end

"""
Compute the gradient of the operator commutativity constraint.
Can be used with descriptor multiplication operator

Parameters
---------------------
C   : (K2,K1) Functional map
op1 : (K1,K1) operator on first basis
op2 : (K2,K2) descriptros on second basis

Output
---------------------
gardient : (K2,K1) gradient of the operator commutativity squared norm
"""
function op_commutation_grad(C, op1, op2)
    return op2' * (op2 * C - C * op1) - (op2 * C - C * op1) * op1'
end


"""
Compute the operator commutativity constraint for a list of pairs of operators
Can be used with a list of descriptor multiplication operator

Parameters
---------------------
C   : (K2,K1) Functional map
op_list : list of tuple( (K1,K1), (K2,K2) ) operators on first and second basis

Output
---------------------
energy : (float) sum of operators commutativity squared norm
"""
function oplist_commutation(C, op_list)
    energy = 0
    for (op1, op2) in op_list
        energy += op_commutation(C, op1, op2)
    end
    return energy
end

"""
Compute the gradient of the operator commutativity constraint for a list of pairs of operators
Can be used with a list of descriptor multiplication operator

Parameters
---------------------
C   : (K2,K1) Functional map
op_list : list of tuple( (K1,K1), (K2,K2) ) operators on first and second basis

Output
---------------------
gradient : (K2,K1) gradient of the sum of operators commutativity squared norm
"""
function oplist_commutation_grad(C, op_list)
    gradient = similar(C)
    fill!(gradient, 0.0)
    for (op1, op2) in op_list
        gradient += op_commutation_grad(C, op1, op2)
    end
    return gradient
end

oplist_commutation_grad

In [81]:
function energy_fn(C_vec)
    C = reshape(C_vec, (k2,k1))
    energy = 0.0
    if descr_μ > 0
        energy += descr_μ * descr_preservation(C, descr1_red, descr2_red)
    end
    if lap_μ > 0
        energy += lap_μ * LB_commutation(C, ev_sqdiff)
    end
    if descr_comm_μ > 0
        energy += descr_comm_μ * oplist_commutation(C, list_descr)
    end
#     if orient_mu > 0:
#         energy += orient_mu * oplist_commutation(C, orient_op)

    return energy
end

"""
Evaluation of the gradient of the energy for standard FM computation

Parameters:
----------------------
C               : (K2*K1) or (K2,K1) Functional map
descr_mu        : scaling of the descriptor preservation term
lap_mu          : scaling of the laplacian commutativity term
descr_comm_mu   : scaling of the descriptor commutativity term
orient_mu       : scaling of the orientation preservation term
descr1          : (K1,p) descriptors on first basis
descr2          : (K2,p) descriptros on second basis
list_descr      : p-uple( (K1,K1), (K2,K2) ) operators on first and second basis
                  related to descriptors.
orient_op       : p-uple( (K1,K1), (K2,K2) ) operators on first and second basis
                  related to orientation preservation operators.
ev_sqdiff       : (K2,K1) [normalized] matrix of squared eigenvalue differences

Output
------------------------
gradient : (K2*K1) - value of the energy
"""
function grad_energy!(g, C)
    C = reshape(C, (k2,k1))
    gradient = similar(C)
    fill!(gradient, 0.0)

    if descr_μ > 0
        gradient += descr_μ * descr_preservation_grad(C, descr1_red, descr2_red)
    end
    
    if lap_μ > 0
        gradient += lap_μ * LB_commutation_grad(C, ev_sqdiff)
    end
    
    if descr_comm_μ > 0
        gradient += descr_comm_μ * oplist_commutation_grad(C, list_descr)
    end
    
#     if orient_mu > 0:
#         gradient += orient_mu * oplist_commutation_grad(C, orient_op)

    gradient[:,1] .= 0
    g .= reshape(gradient, k2*k1)
end

grad_energy!

In [78]:
@time energy_fn(x0)

  0.013195 seconds (9.95 k allocations: 5.222 MiB, 73.97% compilation time)


93.6305232133181

In [76]:
@time grad_energy(x0);

  0.006916 seconds (1.22 k allocations: 11.585 MiB)


In [71]:
using Optim

┌ Info: Precompiling Optim [429524aa-4258-5aef-a3af-852621145aeb]
└ @ Base loading.jl:1313


In [35]:
res = optimize(energy_fn, reshape(x0, k2*k1), LBFGS(), 
    Optim.Options(show_trace=true, iterations = 15, f_tol=1e-4); autodiff=:forward)

Iter     Function value   Gradient norm 
     0     9.363052e+01     6.578420e+01
 * time: 0.012000083923339844
     1     6.010118e+01     2.402624e+01
 * time: 8.934999942779541
     2     4.833240e+01     8.847396e+00
 * time: 22.09599995613098
     3     4.483751e+01     3.880380e+00
 * time: 35.16300010681152
     4     4.293005e+01     3.407950e+00
 * time: 48.914000034332275
     5     4.162311e+01     2.784900e+00
 * time: 61.986000061035156
     6     4.066034e+01     2.649547e+00
 * time: 75.10800004005432
     7     4.007708e+01     1.484537e+00
 * time: 87.99300003051758
     8     3.972626e+01     1.394655e+00
 * time: 100.48399996757507
     9     3.952218e+01     8.998665e-01
 * time: 113.04800009727478
    10     3.937219e+01     7.197819e-01
 * time: 125.85999989509583
    11     3.927948e+01     4.239449e-01
 * time: 138.49699997901917
    12     3.922350e+01     5.746855e-01
 * time: 150.7739999294281
    13     3.918927e+01     2.804708e-01
 * time: 163.950000047683

 * Status: failure (reached maximum number of iterations)

 * Candidate solution
    Final objective value:     3.915634e+01

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 4.80e-03 ≰ 0.0e+00
    |x - x'|/|x'|          = 9.55e-03 ≰ 0.0e+00
    |f(x) - f(x')|         = 1.37e-02 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 3.51e-04 ≰ 1.0e-04
    |g(x)|                 = 2.22e-01 ≰ 1.0e-08

 * Work counters
    Seconds run:   190  (vs limit Inf)
    Iterations:    15
    f(x) calls:    45
    ∇f(x) calls:   45


In [82]:
res = optimize(energy_fn, grad_energy!, reshape(x0, k2*k1), LBFGS(), 
    Optim.Options(show_trace=true, iterations = 15, f_tol=1e-4))

Iter     Function value   Gradient norm 
     0     9.363052e+01     3.184949e+01
 * time: 0.0010001659393310547
     1     7.683701e+01     1.345098e+01
 * time: 0.878000020980835
     2     6.510130e+01     9.666214e+00
 * time: 0.9360001087188721
     3     6.004698e+01     4.484454e+00
 * time: 0.9810001850128174
     4     5.709395e+01     4.370470e+00
 * time: 1.0160000324249268
     5     5.550407e+01     2.511215e+00
 * time: 1.0510001182556152
     6     5.459393e+01     1.985245e+00
 * time: 1.0840001106262207
     7     5.388069e+01     1.735180e+00
 * time: 1.1190001964569092
     8     5.347314e+01     1.487310e+00
 * time: 1.1540000438690186
     9     5.323387e+01     9.660572e-01
 * time: 1.188000202178955
    10     5.307076e+01     7.539892e-01
 * time: 1.2220001220703125
    11     5.296925e+01     5.275951e-01
 * time: 1.2599999904632568
    12     5.290955e+01     4.267190e-01
 * time: 1.2920000553131104
    13     5.287311e+01     2.942810e-01
 * time: 1.327000141

 * Status: failure (reached maximum number of iterations)

 * Candidate solution
    Final objective value:     5.283778e+01

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 4.82e-03 ≰ 0.0e+00
    |x - x'|/|x'|          = 3.88e-03 ≰ 0.0e+00
    |f(x) - f(x')|         = 1.42e-02 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 2.69e-04 ≰ 1.0e-04
    |g(x)|                 = 2.69e-01 ≰ 1.0e-08

 * Work counters
    Seconds run:   1  (vs limit Inf)
    Iterations:    15
    f(x) calls:    45
    ∇f(x) calls:   45


In [128]:
FM_opt = reshape(res.minimizer, (k2,k1));
viz_fm_matrix(FM_opt) |> display

GLMakie.Screen(...)

In [125]:
@show minimum(FM_opt)
@show maximum(FM_opt)

minimum(FM_opt) = -0.5502659816557515
maximum(FM_opt) = 1.2425867898388572


1.2425867898388572

In [45]:
using JLD

In [85]:
save("FM_opt_w_grad.jld", "FM_opt", FM_opt)

In [123]:
FM_opt = load("FM_opt_w_grad.jld", "FM_opt");

In [27]:
py"""
import pyFM.spectral as spectral

def FM_to_p2p(FM_opt, pymesh1, pymesh2):
    return spectral.FM_to_p2p(FM_opt, pymesh1.eigenvectors, pymesh2.eigenvectors)
"""

In [87]:
p2p_map = py"FM_to_p2p"(FM_opt, pymesh1, pymesh2);

In [88]:
py"np.save"("np_FM_w_grad", FM_opt)
py"np.save"("np_p2p_w_grad", p2p_map)

In [112]:
using Statistics

function accuracy(p2p, gt_p2p, D1_geod; return_all=false, sqrt_area=nothing)
    dists = D1_geod[p2p.+1,gt_p2p.+1]
    if sqrt_area != nothing
        dists ./= sqrt_area
    end
    if return_all
        return mean(dists), dists
    end
    return mean(dists)
end

accuracy (generic function with 3 methods)

In [93]:
A_geod = pymesh1.get_geodesic(verbose=true)

100%|██████████| 15/15 [00:41<00:00,  2.73s/it]


7207×7207 Matrix{Float64}:
 0.0         0.0110308   0.0180556   …  0.32673     0.32747     0.0110056
 0.00940081  0.0         0.00951037     0.315326    0.316066    0.0141724
 0.0153161   0.00903885  0.0            0.313852    0.314592    0.0108462
 0.0665902   0.0684021   0.0591114      0.350048    0.350788    0.0587313
 0.0706805   0.0704937   0.0607846      0.346174    0.346914    0.0628848
 0.0860992   0.0865389   0.0773665   …  0.360224    0.360964    0.0786823
 0.0835474   0.0854069   0.0765111      0.364102    0.364841    0.0761916
 0.0164435   0.0213075   0.0143829      0.326108    0.326848    0.00870236
 0.0204266   0.0162888   0.00602857     0.3148      0.31554     0.0142782
 0.0321239   0.029821    0.0189438      0.319083    0.319823    0.0244221
 0.0287384   0.0316397   0.0230293   …  0.32826     0.329       0.0203433
 0.117135    0.119484    0.111651       0.395253    0.395993    0.110956
 0.117423    0.118561    0.110448       0.390022    0.390762    0.11101
 ⋮           

In [115]:
# Load an approximate ground truth map
ground_truth_p2p = Int.(py"np.loadtxt"("../data/lion2cat"));

accuracy(p2p_map, ground_truth_p2p, A_geod, sqrt_area=√(pymesh1.area))

0.5960676531445763

## Misc

In [5]:
# Project the descriptors on the LB basis
function project(py_mesh, func; k=undef)
    eigen_vecs = py_mesh.eigenvectors
    area_matrix = mysparse(py_mesh.A)
    if k==undef
        return eigen_vecs' * area_matrix * func
    elseif k <= size(py_mesh.eigenvectors, 1)
        return eigen_vecs[:,1:k]' * area_matrix * func
    else
         throw(DomainError(k, "At least $k eigenvectors should be computed before projecting"))
    end
end

project (generic function with 1 method)

In [ ]:
# Project the descriptors on the LB basis
descr1_red = project(pymesh1, py_FM_model.descr1; k=k1)
descr2_red = project(pymesh2, py_FM_model.descr2; k=k2);